#  Train GPT-2 Text-Generating Model for Question Generation

Adapted from [Max Woolf](http://minimaxir.com) ([this GitHub repository](https://github.com/minimaxir/gpt-2-simple), [blog post](https://minimaxir.com/2019/09/howto-gpt2/))

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
from google.colab import drive
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import shutil
import json
import tensorflow as tf
from gpt_2_simple.src import model
from tqdm import tqdm

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
#gpt2.download_gpt2(model_name="355M")
#!nvidia-smi

In [0]:
gpt2.mount_gdrive()
drive.mount('/content/drive')   #force_remount=True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def is_mounted():
    """Checks if the Google Drive is mounted."""
    assert os.path.isdir('/content/drive'), "You must mount first using mount_gdrive()"
    
def get_file(file_name):
  is_mounted()
  file_path = "NLU_Project/SummQG/data/squad/sentences_questions_answers/"+file_name 
  print("File path:", file_path)
  shutil.copyfile("/content/drive/My Drive/" + file_path, file_name)

In [0]:
#file_name = "squad_sqa_train.txt"

In [0]:
get_file("squad_sqa_test.txt")

File path: NLU_Project/SummQG/data/squad/sentences_questions_answers/squad_sqa_test.txt


In [0]:
# Previous trained models
# "squad_sqa-15000-0.0001"
# 'squad_sqa-20000-0.0001' # last lost value: 0.46, consistent decreasing 
# "squad_sqa-30000-0.0001" # last lost value: 0.41/0.40  -- this might be 25000, not 30000
# squad_sqa-35000-0.00005 # last lost plateau: 0.29/0.30! 
# squad_sqa-40000-0.00001 # plateu at 0.19! good job!
# squad_sqa-45000-0.000005" #plateuing to 0.18

run_name = "squad_sqa-50000-0.000001"   #at 0.15 before crashing
lr = 0.000001
steps = 5000

In [0]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name="squad_sqa-45000-0.000005",
              steps=steps,
              restore_from = "latest",  #fresh or latest
              learning_rate = lr,
              overwrite = False,
              run_name=run_name,
              print_every=100,
              sample_every=steps,
              save_every=steps
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/squad_sqa-45000-0.000005/model-5000
INFO:tensorflow:Restoring parameters from models/squad_sqa-45000-0.000005/model-5000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:28<00:00, 28.66s/it]


dataset has 6280693 tokens
Training...
[100 | 96.26] loss=0.16 avg=0.16
[200 | 184.28] loss=0.05 avg=0.11
[300 | 272.35] loss=0.19 avg=0.14
[400 | 360.38] loss=0.15 avg=0.14
[500 | 448.39] loss=0.43 avg=0.20
[600 | 536.42] loss=0.12 avg=0.18
[700 | 624.46] loss=0.21 avg=0.19
[800 | 712.48] loss=0.12 avg=0.18
[900 | 800.49] loss=0.10 avg=0.17
[1000 | 888.54] loss=0.18 avg=0.17
[1100 | 976.54] loss=0.10 avg=0.16
[1200 | 1064.54] loss=0.18 avg=0.16
[1300 | 1152.58] loss=0.30 avg=0.18
[1400 | 1240.56] loss=0.11 avg=0.17
[1500 | 1328.54] loss=0.16 avg=0.17
[1600 | 1416.54] loss=0.12 avg=0.17
[1700 | 1504.57] loss=0.14 avg=0.17
[1800 | 1592.55] loss=0.93 avg=0.21
[1900 | 1680.53] loss=0.07 avg=0.20
[2000 | 1768.53] loss=0.08 avg=0.20
[2100 | 1856.55] loss=0.41 avg=0.21
[2200 | 1944.55] loss=0.08 avg=0.20
[2300 | 2032.54] loss=0.11 avg=0.20
[2400 | 2120.59] loss=0.14 avg=0.19
[2500 | 2208.59] loss=0.09 avg=0.19
[2600 | 2296.58] loss=0.13 avg=0.19
[2700 | 2384.58] loss=0.09 avg=0.18
[2800 | 24

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)
print(run_name)

squad_sqa-50000-0.000001


## Load a Trained Model Checkpoint


In [0]:
run_name = "squad_sqa-50000-0.000001"

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
#checkpoint_dir = "checkpoint"
#checkpoint_path = os.path.join(checkpoint_dir, run_name)
#print(checkpoint_path)
#hparams = model.default_hparams()
#with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
#      hparams.override_from_dict(json.load(f))
#ckpt = tf.train.latest_checkpoint(checkpoint_path)

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, checkpoint_dir="/content/checkpoint", run_name=run_name)

Loading checkpoint /content/checkpoint/squad_sqa-50000-0.000001/model-5000
INFO:tensorflow:Restoring parameters from /content/checkpoint/squad_sqa-50000-0.000001/model-5000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
text_file = "squad_sqa_test.txt"

In [0]:
def squad_couplet_data_processing(data_file):
  contexts = []
  gold_questions = []
  with open(data_file, "r") as infile:
    text = infile.read()

  couplet = text.split("<|endoftext|>")
  for i in couplet:
    if i == "" or i == "\n":
      pass
    else:
      c_q = i.split("[question]")
      contexts.append(c_q[0]+"[question] ")
      gold_questions.append(c_q[1][:-1])
  return contexts, gold_questions

In [0]:
contexts, gold_questions = squad_couplet_data_processing(text_file)

In [0]:
contexts[15:19]

['\n<|startoftext|>\n[sentence] Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S.\n[answer] eight\n[question] ',
 '\n<|startoftext|>\n[sentence] The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s.\n[answer] 1920\n[question] ',
 '\n<|startoftext|>\n[sentence] The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s.\n[answer] the College of Science\n[question] ',
 '\n<|startoftext|>\n[sentence] Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, inc

In [0]:
gold_questions[15:19]

[' How many BS level degrees are offered in the College of Engineering at Notre Dame?',
 ' In what year was the College of Engineering at Notre Dame formed?',
 ' Before the creation of the College of Engineering similar studies were carried out at which Notre Dame college?',
 ' How many departments are within the Stinson-Remick Hall of Engineering?']

In [0]:
len(gold_questions)

11877

In [0]:
aa = []
for context in tqdm(contexts[15:19]):
  q = gpt2.generate(sess,
              run_name = run_name,
              length=50,
              temperature=0.9,
              top_k = 50, 
              top_p = 0.9,
              prefix= context,
              nsamples=10,
              batch_size=10,
              include_prefix = False,
              truncate = "<|endoftext|>",
              return_as_list=True
              )
  aa.append(q)


100%|██████████| 4/4 [00:50<00:00, 12.54s/it]


In [0]:
aa

[[' How many B.S. degree programs are offered at Washington University?\n',
  ' How many B.S. programs are offered at Washington University?\n',
  ' How many B.S. degree programs exist at Washington University?\n',
  ' How many B.S. degree programs are offered at Washington University?\n',
  ' How many B.S. schools are within the College of Engineering?\n',
  ' How many B.S. degree programs are offered at Washington University?\n',
  ' How many B.S. programs are offered at Washington University?\n',
  ' How many B.S. programs are offered at Washington University?\n',
  ' How many B.S. degree programs are offered at Washington University?\n',
  ' How many B.S. exist in the school of applied sciences?\n'],
 [' When was the College of Engineering established?\n',
  ' When was the College of Engineering established?\n',
  ' When was the College of Engineering established?\n',
  ' When was the College of Engineering established?\n',
  ' When was the College of Engineering established?\n',
 

## Bulk Generation

In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple

In [1]:
%tensorflow_version 1.x
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import shutil
import json
import tensorflow as tf
from gpt_2_simple.src import model
from tqdm import tqdm

run_name = "squad_sqa-50000-0.000001"
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, checkpoint_dir="/content/checkpoint", run_name=run_name)

text_file = "squad_sqa_test.txt"

def squad_couplet_data_processing(data_file):
  contexts = []
  gold_questions = []
  with open(data_file, "r") as infile:
    text = infile.read()

  couplet = text.split("<|endoftext|>")
  for i in couplet:
    if i == "" or i == "\n":
      pass
    else:
      c_q = i.split("[question]")
      contexts.append(c_q[0]+"[question] ")
      gold_questions.append(c_q[1][:-1])
  return contexts, gold_questions

contexts, gold_questions = squad_couplet_data_processing(text_file)

###############################################################################
generated_questions = {}
i = 9900

for context in tqdm(contexts[i:i+100]):
  q = gpt2.generate(sess,
              run_name = run_name,
              length=50,
              temperature=0.1,
              top_k = 50, 
              top_p = 0.9,
              prefix= context,
              nsamples=10,
              batch_size=10,
              include_prefix = False,
              truncate = "<|endoftext|>",
              return_as_list=True
              )
  generated_questions[i] = q
  i += 1

output_file = "squad_sqa_"+str(i)+".json"
with open(output_file, "w") as json_file:
  json.dump(generated_questions, json_file)
shutil.copyfile(output_file, "/content/drive/My Drive/NLU_Project/SummQG/results/squad_sqa/" + output_file)

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading checkpoint /content/checkpoint/squad_sqa-50000-0.000001/model-5000
INFO:tensorflow:Restoring parameters from /content/checkpoint/squad_sqa-50000-0.000001/model-5000


  0%|          | 0/100 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100%|██████████| 100/100 [40:44<00:00, 24.44s/it]


'/content/drive/My Drive/NLU_Project/SummQG/results/squad_sqa/squad_sqa_10000.json'

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.